In [1]:
# main_script.py

import mlflow
import mlflow.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

def load_and_preprocess_data():
    iris = load_iris()
    X, y = iris.data, iris.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)

    return X_train, X_test, y_train, y_test

def create_model(input_shape, hidden_size, num_classes):
    model = Sequential([
        Dense(hidden_size, activation='relu', input_shape=input_shape),
        Dense(num_classes, activation='softmax')
    ])
    return model

def train_model(model, X_train, y_train, X_val, y_val, optimizer, batch_size, epochs):
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                        batch_size=batch_size, epochs=epochs, verbose=0)
    return history

def run_experiment(config):
    mlflow.set_experiment("iris_tf_classification")

    with mlflow.start_run():
        mlflow.log_params(config)
        
        X_train, X_test, y_train, y_test = load_and_preprocess_data()
        
        model = create_model((4,), config["hidden_size"], 3)
        optimizer = Adam(learning_rate=config["learning_rate"])
        
        X_train_split, X_val, y_train_split, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42)
        
        history = train_model(model, X_train_split, y_train_split, X_val, y_val,
                              optimizer, config["batch_size"], config["epochs"])
        
        train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
        
        mlflow.log_metric("final_train_loss", train_loss)
        mlflow.log_metric("final_train_accuracy", train_accuracy)
        mlflow.log_metric("final_test_loss", test_loss)
        mlflow.log_metric("final_test_accuracy", test_accuracy)
        
        mlflow.keras.log_model(model, "model")
        
        for epoch in range(config["epochs"]):
            mlflow.log_metrics({
                "train_loss": history.history['loss'][epoch],
                "train_accuracy": history.history['accuracy'][epoch],
                "val_loss": history.history['val_loss'][epoch],
                "val_accuracy": history.history['val_accuracy'][epoch]
            }, step=epoch)
        
        summary = f"Experiment Summary:\n"
        summary += f"  Config: {config}\n"
        summary += f"  Final Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}\n"
        summary += f"  Final Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}\n"
        summary += f"  Run ID: {mlflow.active_run().info.run_id}\n"
        
        with open("experiment_summary.txt", "w") as f:
            f.write(summary)
        mlflow.log_artifact("experiment_summary.txt")
        
        print(summary)

    return train_accuracy, test_accuracy

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# experiment_1.py
config = {
    "hidden_size": 10,
    "learning_rate": 0.01,
    "batch_size": 32,
    "epochs": 100
}

train_accuracy, test_accuracy = run_experiment(config)

print(f"Experiment completed. Final train accuracy: {train_accuracy:.4f}, test accuracy: {test_accuracy:.4f}")

2024/08/16 16:44:52 INFO mlflow.tracking.fluent: Experiment with name 'iris_tf_classification' does not exist. Creating a new experiment.
2024/08/16 16:44:58 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmp22hm99ya/model/data/model/assets


/usr/local/lib/python3.8/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/08/16 16:45:03 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Experiment Summary:
  Config: {'hidden_size': 10, 'learning_rate': 0.01, 'batch_size': 32, 'epochs': 100}
  Final Train Loss: 0.0710, Accuracy: 0.9833
  Final Test Loss: 0.1058, Accuracy: 0.9667
  Run ID: a54a6a72e68f40a4b78e58a8ef07b938

Experiment completed. Final train accuracy: 0.9833, test accuracy: 0.9667


In [ ]:
# experiment_1.py
config = {
    "hidden_size": 10,
    "learning_rate": 0.01,
    "batch_size": 32,
    "epochs": 100
}

train_accuracy, test_accuracy = run_experiment(config)

print(f"Experiment completed. Final train accuracy: {train_accuracy:.4f}, test accuracy: {test_accuracy:.4f}")

In [3]:
# Experiment configurations
experiments = [
    {"hidden_size": 20, "learning_rate": 0.001, "batch_size": 64, "epochs": 200},
    {"hidden_size": 15, "learning_rate": 0.005, "batch_size": 32, "epochs": 150},
    {"hidden_size": 25, "learning_rate": 0.0005, "batch_size": 64, "epochs": 250},
    {"hidden_size": 30, "learning_rate": 0.0001, "batch_size": 32, "epochs": 300}
]

for config in experiments:
    train_accuracy, test_accuracy = run_experiment(config)

    print(f"Experiment completed. Final train accuracy: {train_accuracy:.4f}, test accuracy: {test_accuracy:.4f}")

2024/08/16 16:47:12 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmphq1gl8m1/model/data/model/assets


2024/08/16 16:47:15 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Experiment Summary:
  Config: {'hidden_size': 20, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 200}
  Final Train Loss: 0.2887, Accuracy: 0.9083
  Final Test Loss: 0.2311, Accuracy: 0.9667
  Run ID: 5b82482d34de470b8804a6a2c19317f8

Experiment completed. Final train accuracy: 0.9083, test accuracy: 0.9667


2024/08/16 16:47:22 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


...

Experiment Summary:
  Config: {'hidden_size': 25, 'learning_rate': 0.0005, 'batch_size': 64, 'epochs': 250}
  Final Train Loss: 0.2854, Accuracy: 0.8667
  Final Test Loss: 0.2274, Accuracy: 0.9333
  Run ID: cceef0e764f04b79a32e25446283e3cd

Experiment completed. Final train accuracy: 0.8667, test accuracy: 0.9333


2024/08/16 16:47:50 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmprgqp5y88/model/data/model/assets


2024/08/16 16:47:53 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Experiment Summary:
  Config: {'hidden_size': 30, 'learning_rate': 0.0001, 'batch_size': 32, 'epochs': 300}
  Final Train Loss: 0.5925, Accuracy: 0.8583
  Final Test Loss: 0.5666, Accuracy: 0.9000
  Run ID: 9379a54ebe9b4347acac6186372a0376

Experiment completed. Final train accuracy: 0.8583, test accuracy: 0.9000
